## Install needed libraries

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/vnquant_package"
%cd {path}
!ls

Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/vnquant_package'
/content
gdrive	sample_data


In [ ]:
!git clone https://github.com/phamdinhkhanh/vnquant
%cd vnquant
!python setup.py install

Cloning into 'vnquant'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 341 (delta 65), reused 89 (delta 53), pack-reused 237
Receiving objects: 100% (341/341), 1.05 MiB | 12.24 MiB/s, done.
Resolving deltas: 100% (164/164), done.
/content/vnquant
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-package

In [ ]:
%cd ..
!rm -rf vnquant
!ls

/content
gdrive	sample_data


In [ ]:
!pip freeze | grep vnquant

vnquant==0.1.1


In [ ]:
import vnquant
vnquant.__version__

'0.1.1'

## Import libraries

In [ ]:
import vnquant.data as dt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import accuracy_score, classification_report

## We choose 10 codes from biggest companies on stock exchange in Vietnam

In [ ]:
stock_symbols = ['VIC', 'VCB', 'VNM', 'GAS', 'HVN', 'MWG', 'FPT', 'HPG', 'MSN', 'SAB']

## Prepare and process the dataset

In [ ]:
stock = {}
for i in range(len(stock_symbols)):
  data = dt.DataLoader(symbols=stock_symbols[i],
                        start="2014-01-01",
                        end="2024-01-01",
                        data_source="VND")
  stock[stock_symbols[i]] = data.download()

In [ ]:
stock[stock_symbols[0]]

Attributes,high,low,open,close,avg,volume
Symbols,VIC,VIC,VIC,VIC,VIC,VIC
date,,,,,,
2014-01-02,70.50,69.50,70.00,70.00,69.92,74950.0
2014-01-03,70.50,69.50,70.00,70.50,70.00,91920.0
2014-01-06,70.50,69.50,70.50,70.50,70.11,111760.0
2014-01-07,70.50,70.00,70.50,70.50,70.47,294120.0
2014-01-08,70.50,70.00,70.00,70.00,70.02,109890.0
...,...,...,...,...,...,...
2023-12-25,43.55,43.00,43.10,43.40,43.34,3364500.0
2023-12-26,43.75,43.35,43.40,43.55,43.55,1806700.0


In [ ]:
for i in range(len(stock_symbols)):
  print(f"Length of {stock_symbols[i]} with index {i}: ", len(stock[stock_symbols[i]]))

Length of VIC with index 0:  2494
Length of VCB with index 1:  2494
Length of VNM with index 2:  2494
Length of GAS with index 3:  2494
Length of HVN with index 4:  1743
Length of MWG with index 5:  2368
Length of FPT with index 6:  2494
Length of HPG with index 7:  2494
Length of MSN with index 8:  2494
Length of SAB with index 9:  1767


In [ ]:
changed_start_index5 = len(stock[stock_symbols[5]]) - len(stock[stock_symbols[4]])
stock[stock_symbols[5]] = stock[stock_symbols[5]][changed_start_index5:]

In [ ]:
changed_start_index9 = len(stock[stock_symbols[9]]) - len(stock[stock_symbols[4]])
stock[stock_symbols[9]] = stock[stock_symbols[9]][changed_start_index9:]

In [ ]:
changed_start_other_index = len(stock[stock_symbols[0]]) - len(stock[stock_symbols[4]])
for i in range(len(stock_symbols)):
  if ((i == 4) or (i == 5) or (i == 9)):
    pass
  else:
    stock[stock_symbols[i]] = stock[stock_symbols[i]][changed_start_other_index:]

In [ ]:
combined_high = (stock[stock_symbols[0]]['high'].values + stock[stock_symbols[1]]['high'].values + stock[stock_symbols[2]]['high'].values +
                 stock[stock_symbols[3]]['high'].values + stock[stock_symbols[4]]['high'].values + stock[stock_symbols[5]]['high'].values +
                 stock[stock_symbols[6]]['high'].values + stock[stock_symbols[7]]['high'].values + stock[stock_symbols[8]]['high'].values +
                 stock[stock_symbols[9]]['high'].values) / len(stock_symbols)
combined_high

array([[82.885],
       [83.95 ],
       [85.11 ],
       ...,
       [58.545],
       [58.44 ],
       [58.635]])

In [ ]:
combined_low = (stock[stock_symbols[0]]['low'].values + stock[stock_symbols[1]]['low'].values + stock[stock_symbols[2]]['low'].values +
                 stock[stock_symbols[3]]['low'].values + stock[stock_symbols[4]]['low'].values + stock[stock_symbols[5]]['low'].values +
                 stock[stock_symbols[6]]['low'].values + stock[stock_symbols[7]]['low'].values + stock[stock_symbols[8]]['low'].values +
                 stock[stock_symbols[9]]['low'].values) / len(stock_symbols)
combined_low

array([[81.455],
       [82.445],
       [83.565],
       ...,
       [57.755],
       [57.685],
       [57.625]])

In [ ]:
combined_open = (stock[stock_symbols[0]]['open'].values + stock[stock_symbols[1]]['open'].values + stock[stock_symbols[2]]['open'].values +
                 stock[stock_symbols[3]]['open'].values + stock[stock_symbols[4]]['open'].values + stock[stock_symbols[5]]['open'].values +
                 stock[stock_symbols[6]]['open'].values + stock[stock_symbols[7]]['open'].values + stock[stock_symbols[8]]['open'].values +
                 stock[stock_symbols[9]]['open'].values) / len(stock_symbols)
combined_open

array([[82.255],
       [82.795],
       [84.76 ],
       ...,
       [58.055],
       [58.01 ],
       [58.325]])

In [ ]:
combined_close = (stock[stock_symbols[0]]['close'].values + stock[stock_symbols[1]]['close'].values + stock[stock_symbols[2]]['close'].values +
                 stock[stock_symbols[3]]['close'].values + stock[stock_symbols[4]]['close'].values + stock[stock_symbols[5]]['close'].values +
                 stock[stock_symbols[6]]['close'].values + stock[stock_symbols[7]]['close'].values + stock[stock_symbols[8]]['close'].values +
                 stock[stock_symbols[9]]['close'].values) / len(stock_symbols)
combined_close

array([[82.185],
       [83.725],
       [84.06 ],
       ...,
       [57.93 ],
       [58.09 ],
       [57.71 ]])

In [ ]:
combined_avg = (stock[stock_symbols[0]]['avg'].values + stock[stock_symbols[1]]['avg'].values + stock[stock_symbols[2]]['avg'].values +
                 stock[stock_symbols[3]]['avg'].values + stock[stock_symbols[4]]['avg'].values + stock[stock_symbols[5]]['avg'].values +
                 stock[stock_symbols[6]]['avg'].values + stock[stock_symbols[7]]['avg'].values + stock[stock_symbols[8]]['avg'].values +
                 stock[stock_symbols[9]]['avg'].values) / len(stock_symbols)
combined_avg

array([[82.107 ],
       [83.28  ],
       [84.3326],
       ...,
       [58.148 ],
       [58.099 ],
       [58.183 ]])

In [ ]:
combined_volume = (stock[stock_symbols[0]]['volume'].values + stock[stock_symbols[1]]['volume'].values + stock[stock_symbols[2]]['volume'].values +
                 stock[stock_symbols[3]]['volume'].values + stock[stock_symbols[4]]['volume'].values + stock[stock_symbols[5]]['volume'].values +
                 stock[stock_symbols[6]]['volume'].values + stock[stock_symbols[7]]['volume'].values + stock[stock_symbols[8]]['volume'].values +
                 stock[stock_symbols[9]]['volume'].values) / len(stock_symbols)
combined_volume

array([[ 747272. ],
       [ 605305.8],
       [ 643985. ],
       ...,
       [4714230. ],
       [5987070. ],
       [5748880.1]])

In [ ]:
df = pd.DataFrame(combined_high)
df.head()

,0
0,82.885
1,83.950
2,85.110
3,84.750
4,83.620


In [ ]:
df = df.rename(columns={0: 'high'})

df['low'] = combined_low
df['open'] = combined_open
df['close'] = combined_close
df['avg'] = combined_avg
df['volume'] = combined_volume

df.head()

,high,low,open,close,avg,volume
0,82.885,81.455,82.255,82.185,82.1070,747272.0
1,83.950,82.445,82.795,83.725,83.2800,605305.8
2,85.110,83.565,84.760,84.060,84.3326,643985.0
3,84.750,83.060,84.335,83.335,83.6911,727448.6
4,83.620,82.000,83.465,82.270,82.5046,722580.3


In [ ]:
def reconstruct_data(list_price):
  week = []
  combine = []
  i = 0
  while (i < len(list_price)):
    if (len(week) != 4):
      week.append(np.round(np.mean(list_price[i:i+7]), 2))
      i = i + 7
    else:
      combine.append(week)
      week = []

  return combine

In [ ]:
close = df['close'].values
open = df['open'].values
high = df['high'].values
low = df['low'].values
avg = df['avg'].values
volume = df['volume'].values

In [ ]:
reconstruct_close = reconstruct_data(close)

In [ ]:
df_combined = pd.DataFrame(reconstruct_close, columns =['close_week1', 'close_week2', 'close_week3', 'close_week4'], dtype = np.dtype(np.float64))
df_combined.head()

,close_week1,close_week2,close_week3,close_week4
0,83.20,82.67,82.90,83.42
1,83.14,81.65,81.39,80.47
2,80.35,78.91,79.28,79.41
3,74.30,71.38,72.74,74.19
4,75.26,75.92,77.06,78.61


In [ ]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [ ]:
df_combined['close_total4week'] = mean_close[:len(df_combined)]
df_combined['open_total4week'] = mean_open[:len(df_combined)]
df_combined['high_total4week'] = mean_high[:len(df_combined)]
df_combined['low_total4week'] = mean_low[:len(df_combined)]
df_combined['avg_total4week'] = mean_avg[:len(df_combined)]
df_combined['volume_total4week'] = mean_volume[:len(df_combined)]

In [ ]:
df_combined

,close_week1,close_week2,close_week3,close_week4,close_total4week,open_total4week,high_total4week,low_total4week,avg_total4week,volume_total4week
0,83.20,82.67,82.90,83.42,83.05,83.02,83.80,82.26,83.00,894676.19
1,83.14,81.65,81.39,80.47,81.66,81.80,82.53,80.98,81.71,1087039.61
2,80.35,78.91,79.28,79.41,79.49,79.50,80.12,78.91,79.48,859013.45
3,74.30,71.38,72.74,74.19,73.15,73.05,73.69,72.48,73.06,1091261.38
4,75.26,75.92,77.06,78.61,76.71,76.51,77.19,75.82,76.64,924784.75
...,...,...,...,...,...,...,...,...,...,...
57,71.41,71.34,71.89,71.45,71.52,71.53,72.16,70.94,71.49,4757622.29
58,73.45,74.37,75.42,74.49,74.43,74.52,75.17,73.74,74.43,5715012.70
59,74.80,77.14,67.84,62.91,70.67,70.85,71.66,69.83,70.69,6899552.72
60,61.36,60.05,56.05,57.37,58.71,58.85,59.51,57.90,58.66,5045590.63


In [ ]:
df_combined = df_combined.ewm(alpha=0.65).mean()

In [ ]:
df_combined['ema50'] = df_combined['close_total4week'] / df_combined['close_total4week'].ewm(50).mean()
df_combined['ema21'] = df_combined['close_total4week'] / df_combined['close_total4week'].ewm(21).mean()
df_combined['ema14'] = df_combined['close_total4week'] / df_combined['close_total4week'].ewm(14).mean()
df_combined['ema5'] = df_combined['close_total4week'] / df_combined['close_total4week'].ewm(5).mean()

In [ ]:
def rsi(X, window=14):
  delta = X.diff(1)

  gains = delta.where(delta > 0, 0)
  losses = -delta.where(delta < 0, 0)

  avg_gains = gains.rolling(window=window, min_periods=1).mean()
  avg_losses = losses.rolling(window=window, min_periods=1).mean()

  rs = avg_gains / avg_losses
  rsi = 100 - (100 / (1 + rs))

  return rsi

In [ ]:
df_combined['rsi'] = rsi(df_combined['close_total4week'])

In [ ]:
def macd(X, short_window=12, long_window=29, signal=9):
  short_ema = X.ewm(span=short_window, adjust=False).mean()

  long_ema = X.ewm(span=long_window, adjust=False).mean()

  macd = short_ema - long_ema

  signal = macd.ewm(span=signal, adjust=False).mean()

  return signal

In [ ]:
df_combined['macd'] = macd(df_combined['close_total4week'])

In [ ]:
def obv(X):
  obv = pd.Series(index=X.index)
  obv.iloc[0] = 0

  for i in range(1, len(X)):
    if (X['close_total4week'].iloc[i] > X['close_total4week'].iloc[i-1]):
      obv.iloc[i] = obv.iloc[i-1] + X['volume_total4week'].iloc[i]
    elif (X['close_total4week'].iloc[i] < X['close_total4week'].iloc[i-1]):
      obv.iloc[i] = obv.iloc[i-1] - X['volume_total4week'].iloc[i]
    else:
      obv.iloc[i] = obv.iloc[i-1]

  return obv

In [ ]:
df_combined['obv'] = obv(df_combined)

In [ ]:
def emv(X):
  emv = pd.Series(index=X.index)
  emv.iloc[0] = np.nan

  for i in range(1, len(X)):
    dm = 0.5 * ((X['high_total4week'].iloc[i] + X['low_total4week'].iloc[i]) - (X['high_total4week'].iloc[i-1] + X['low_total4week'].iloc[i-1]))
    br = X['volume_total4week'].iloc[i] / (1000000 * (X['high_total4week'].iloc[i] - X['low_total4week'].iloc[i]))
    emv.iloc[i] = dm / br if br != 0 else 0

  return emv

In [ ]:
df_combined['emv'] = emv(df_combined)

In [ ]:
def mfi(X, window=14):
  combine_price = (X['high_total4week'] + X['low_total4week'] + X['close_total4week']) / 3

  raw_money_flow = combine_price * X['volume_total4week']

  flow_direction = (combine_price.diff() > 0).astype(int)

  positive_money_flow = flow_direction * raw_money_flow
  negative_money_flow = (1 - flow_direction) * raw_money_flow

  positive = positive_money_flow.rolling(window=window, min_periods=1).sum()
  negative = negative_money_flow.rolling(window=window, min_periods=1).sum()

  mf = positive / negative
  mfi = 100 - (100 / (1 + mf))

  return mfi

In [ ]:
df_combined['mfi'] = mfi(df_combined)

In [ ]:
df_combined['volume_total4week'] = df_combined['volume_total4week'] / df_combined['volume_total4week'].ewm(5).mean()

In [ ]:
def closediff(x, y):
  res = []

  for i in range(len(x)):
    res.append(((y[i]-x[i]) / x[i])*100)

  return res

In [ ]:
close_week1 = df_combined['close_week1'].values
close_week2 = df_combined['close_week2'].values
close_week3 = df_combined['close_week3'].values
close_week4 = df_combined['close_week4'].values

df_combined['closediff_week2_1'] = closediff(close_week1, close_week2)
df_combined['closediff_week3_1'] = closediff(close_week1, close_week3)
df_combined['closediff_week4_1'] = closediff(close_week1, close_week4)
df_combined['closediff_week3_2'] = closediff(close_week2, close_week3)
df_combined['closediff_week4_2'] = closediff(close_week2, close_week4)
df_combined['closediff_week4_3'] = closediff(close_week3, close_week4)

In [ ]:
def total_closediff(a, b, c, d, e, f):
  res = []

  for i in range(len(a)):
    res.append(((a[i]+b[i]+c[i]+d[i]+e[i]+f[i])/6))

  return res

In [ ]:
df_combined['total_closediff'] = total_closediff(df_combined['closediff_week2_1'], df_combined['closediff_week3_1'],
                                                  df_combined['closediff_week4_1'], df_combined['closediff_week3_2'],
                                                    df_combined['closediff_week4_2'], df_combined['closediff_week4_3'])

In [ ]:
df_combined.replace(0, np.nan, inplace=True)
df_combined = df_combined.dropna()

In [ ]:
df_combined['trend'] = (df_combined['total_closediff'].iloc[:] > 0).astype(int)

In [ ]:
df_combined

,close_week1,close_week2,close_week3,close_week4,close_total4week,open_total4week,high_total4week,low_total4week,avg_total4week,volume_total4week,...,emv,mfi,closediff_week2_1,closediff_week3_1,closediff_week4_1,closediff_week3_2,closediff_week4_2,closediff_week4_3,total_closediff,trend
4,75.746499,75.347751,76.428285,77.762451,76.319196,76.170068,76.838499,75.511188,76.252983,0.989192,...,0.859013,19.078798,-0.526425,0.900089,2.661446,1.434063,3.204741,1.745645,1.569926,1
5,78.555438,78.234019,79.314205,82.293187,79.601753,79.393449,80.297333,78.668720,79.513538,0.917050,...,6.209720,31.396573,-0.409163,0.965899,4.758103,1.380712,5.188495,3.755925,2.606662,1
6,83.046293,82.615225,83.779845,86.765993,84.055979,83.957143,84.865005,83.131423,84.051158,1.062894,...,7.636524,42.349734,-0.519069,0.883305,4.479068,1.409691,5.024216,3.564280,2.473582,1
7,88.015822,89.197811,92.660946,97.814586,91.925365,91.539708,92.936569,90.464149,91.826156,1.247818,...,14.737173,52.740245,1.342929,5.277601,11.132958,3.882533,9.660298,5.561825,6.143024,1
8,99.277425,99.262027,99.245350,104.461629,100.561058,100.439099,101.994013,99.081131,100.526340,1.328197,...,17.114961,61.572247,-0.015510,-0.032309,5.221936,-0.016801,5.238259,5.255943,2.608586,1
9,106.000284,108.094453,103.857000,107.554655,106.378031,106.237845,108.023571,104.547047,106.268378,1.435838,...,10.960803,68.997886,1.975626,-2.021961,1.466384,-3.920140,-0.499376,3.560334,0.093478,1
10,108.593625,110.386081,111.217021,112.199174,110.597852,110.529287,112.330791,108.897508,110.585474,1.285854,...,8.482544,74.017932,1.650609,2.415792,3.320222,0.752758,1.642502,0.883096,1.777497,1
11,111.145777,107.730119,107.247444,108.345198,108.616741,108.943745,110.646771,106.838621,108.716410,1.144838,...,-4.415702,64.554963,-3.073133,-3.507406,-2.519735,-0.448041,0.570944,1.023571,-1.325633,0
12,103.387513,105.208039,103.043600,101.705311,103.334353,103.234304,105.045863,101.353511,103.200237,1.088217,...,-13.075742,57.753637,1.760876,-0.332644,-1.627084,-2.057294,-3.329334,-1.298760,-1.147373,0
13,95.985626,96.323810,96.248757,96.462857,96.259521,96.192003,97.658049,94.649726,96.154080,0.979934,...,-15.104612,53.082360,0.352327,0.274136,0.497189,-0.077917,0.144354,0.222444,0.235422,1


In [ ]:
df_combined = df_combined[['ema50', 'ema21', 'ema14', 'ema5', 'volume_total4week', 'rsi', 'macd', 'obv', 'emv', 'mfi', 'closediff_week2_1',
                           'closediff_week3_1', 'closediff_week4_1', 'closediff_week3_2', 'closediff_week4_2', 'closediff_week4_3', 'total_closediff', 'trend']]

## LSTM Model

In [ ]:
X = df_combined.loc[:, df_combined.columns != ('trend')]
y = df_combined.loc[:, df_combined.columns == ('trend')]
scale = MinMaxScaler(feature_range=(0,1))
X_scaled = scale.fit_transform(X)

timesteps = 10
feature = X_scaled.shape[1]

test_len = int(len(X_scaled)*0.2)
train_len = X_scaled.shape[0] - test_len

X_train = []
y_train = []


for i in range(timesteps, train_len):
  X_train.append(X_scaled[i-timesteps:i,:])
  y_train.append(y.values[i,:])



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

print("X train dataset shape",X_train.shape)
print("y train dataset shape",y_train.shape)

X_test = []
y_test = []

for i in range(train_len-1, len(X_scaled)):
  X_test.append(X_scaled[i-timesteps: i, :])
  y_test.append(y.values[i,:])

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print("X test dataset shape",X_test.shape)
print("y test dataset shape",y_test.shape)

X train dataset shape (37, 10, 17)
y train dataset shape (37, 1)
X test dataset shape (12, 10, 17)
y test dataset shape (12, 1)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128,return_sequences=True,input_shape=(timesteps, feature)))
lstm_model.add(LSTM(64, input_shape=(timesteps, feature)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
lstm_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 10, 128)           74752     
                                                                 
 lstm_29 (LSTM)              (None, 64)                49408     
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124225 (485.25 KB)
Trainable params: 124225 (485.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Epoch 1/15
5/5 [==============================] - 5s 21ms/step - loss: 0.2580
Epoch 2/15
5/5 [==============================] - 0s 22ms/step - loss: 0.2408
Epoch 3/15
5/5 [==============================] - 0s 22ms/step - loss: 0.2369
Epoch 4/15
5/5 [==============================] - 0s 23ms/step - loss: 0.2374
Epoch 5/15
5/5 [==============================] - 0s 23ms/step - loss: 0.2285
Epoch 6/15
5/5 [==============================] - 0s 24ms/step - loss: 0.2225
Epoch 7/15
5/5 [==============================] - 0s 23ms/step - loss: 0.2130
Epoch 8/15
5/5 [==============================] - 0s 21ms/step - loss: 0.2099
Epoch 9/15
5/5 [==============================] - 0s 23ms/step - loss: 0.2172
Epoch 10/15
5/5 [==============================] - 0s 22ms/step - loss: 0.2006
Epoch 11/15
5/5 [==============================] - 0s 20ms/step - loss: 0.1937
Epoch 12/15
5/5 [==============================] - 0s 22ms/step - loss: 0.1848
Epoch 13/15
5/5 [==============================] - 0s 23ms/st

In [ ]:
y_pred = lstm_model.predict(X_test, batch_size=4)
prediction = (y_pred > 0.5).astype(int)

3/3 [==============================] - 1s 7ms/step


In [ ]:
accuracy = accuracy_score(prediction, y_test)
classification_rep = classification_report(prediction, y_test)
print("Accuracy: ", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy:  0.75
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.33      0.50      0.40         2

    accuracy                           0.75        12
   macro avg       0.61      0.65      0.62        12
weighted avg       0.80      0.75      0.77        12



## THE END